
# 📥 Torrent to GDrive Downloader

**Updated: 2025/08/17**

This project allows you to **download torrent files or magnet links directly into your Google Drive** using Google Colab.  
- You can add **one or more torrents**, view their details, reorder them, and change download settings.  
- The files are saved straight into your **Google Drive**, so you don't need to worry about local storage.  
- Please make sure you select the **T4 GPU runtime** in Google Colab before starting. This way, you will be able to download a cumulative size of approx. 384GiB in one session.

👉 Even if you don't understand code/output logs, just follow the instructions step by step and run each cell accordingly.  

**⚠️ Dev's Note on multiple torrent downloads:** The current code sets active downloads to 1, that is, it will download torrents sequentially. An update to enable parallel downloading/user preference setting will be added soon.

I am working on a fix. In the meantime, it is recommended to **upload and download only one torrent at a time** and restart the runtime before downloading the next torrent.

**📃 Credits:** This project takes help from multiple existing Torrent to Google Drive Downloader scripts avaiable on the internet, which no longer seem to work. The dependency of this script is [libtorrent](https://www.google.com/url?q=https%3A%2F%2Fwww.libtorrent.org%2F) library, and this script needs to be updated as an when anything gets changed in the library.

## 🔹 Cell 1: Importing Required Libraries

In this section, the script installs and imports the necessary libraries (especially **libtorrent**) that handle torrents.  

👉 You don’t need to worry about the log messages here. As long as **libtorrent installs successfully**, everything is fine.  

In [ ]:
!python -m pip install --upgrade pip setuptools wheel
!pip install libtorrent==2.0.11
import libtorrent as lt
import time
from google.colab import files

# Initialize libtorrent session
ses = lt.session()
ses.listen_on(6881, 6891)

settings = ses.get_settings()
settings['active_downloads'] = 1      # Only 1 torrent downloading at a time
ses.apply_settings(settings)

## 🔹 Cell 2: Adding Torrents

Here, you can provide the torrents you want to download.  

 You will see a menu:  
 1. **Upload .torrent file(s)** → You can upload torrent files from your computer.  
 2. **Paste magnet link(s)** → You can paste one or more magnet links.  
    - Paste links one by one.  
    - When finished, press **Enter** on an empty line.  

 You can repeat the process to add both torrent files and magnet links.  
 When done, type **exit** to stop adding.  

 At the end of this section, you will see how many torrents were successfully added.  

In [ ]:
# Store torrent data before adding to session
torrent_data = []  # Will store {'type': 'file'/'magnet', 'data': filename/magnet_uri}

def add_torrent_file():
    source = files.upload()
    for filename in source.keys():
        try:
            # Just validate the torrent file, don't add to session yet
            ti = lt.torrent_info(filename)
            torrent_data.append({'type': 'file', 'data': filename, 'name': ti.name()})
            print(f"Added torrent file: {ti.name()}")
        except Exception as e:
            print(f"Error reading torrent file {filename}: {e}")

def add_magnet_links():
    print("Paste magnet links one per line. Press Enter on an empty line to finish.")
    while True:
        magnet_link = input("Enter magnet link: ").strip()
        if not magnet_link:
            break
        try:
            # Extract name from magnet link if possible
            import urllib.parse
            parsed = urllib.parse.parse_qs(magnet_link.split('?')[1] if '?' in magnet_link else '')
            name = parsed.get('dn', ['Unknown Torrent'])[0]
            torrent_data.append({'type': 'magnet', 'data': magnet_link, 'name': name})
            print(f"Added magnet: {name}")
        except Exception as e:
            print(f"Error adding magnet link: {e}")

print("Choose input type:")
print("1. Upload .torrent file(s)")
print("2. Paste magnet link(s)")

while True:
    choice = input("Enter choice (1/2) or 'exit' to finish adding: ").strip().lower()

    if choice == "1":
        add_torrent_file()
    elif choice == "2":
        add_magnet_links()
    elif choice == "exit":
        break
    else:
        print("Invalid choice.")

print(f"\nAdded {len(torrent_data)} torrents to queue.")

## 🔹 Cell 3: Fetching Metadata & Modifying Settings

 In this section:  
 - A popup will appear to connect your **Google Drive**. Please select the **correct Google account** where you want your files to be saved.  
 - The script will fetch details (metadata) about each torrent.  

 📌 Once metadata is ready, you'll see a **settings menu**.  
 👉 If you don't want to change anything, you can simply choose **Option 1** to continue with the default settings and download in the same order as added.  

 Here's what the menu options mean:  
 1. **Confirm Settings and Exit Menu** → Continue with current settings and start downloads.  
 2. **Change Save Path** → Choose where in your Google Drive to save the files.  
 3. **View Torrent Metadata** → See details (file list and sizes) for each torrent.  
 4. **Remove Torrent** → Remove a torrent you don't want to download.  
 5. **Reorder Torrents** → Change the order in which torrents will be downloaded.  
 6. **Change Timeout Setting** → Adjust how long the program waits before deciding a torrent is stuck.  
 7. **Change Ask After Each Download Setting** → Choose whether you want the program to pause and ask you after each download finishes.  

 ⚠️ By default:  
 - Files are saved in **`/My Drive/__TorDownSave`**.  
 - Timeout is set to **10 minutes** per torrent.  
 - It does **not** ask after each download.  

In [ ]:
import ipywidgets as widgets
from IPython.display import display, clear_output
import sys
import shutil
import os

# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

# Define all default settings
default_save_path = "/content/drive/My Drive/__TorDownSave"
DEFAULT_TIMEOUT = 600  # seconds (10 minutes)
ASK_AFTER_EACH = False
save_path = default_save_path

print(f"\nDrive mounted. Default save path: {save_path}")

# Check available Google Drive space
def get_free_space_gb(path="/content/drive"):
    total, used, free = shutil.disk_usage(path)
    return free / (1024 ** 3)

free_gb = get_free_space_gb()

# Now add torrents to session and fetch metadata
valid_torrent_infos = []  # Will store (index, torrent_info, handle) tuples
METADATA_TIMEOUT = 60

print("\nAdding torrents to session and fetching metadata...")

for index, torrent in enumerate(torrent_data):
    try:
        print(f"Processing torrent {index + 1}/{len(torrent_data)}: {torrent['name']}")

        # Add to libtorrent session
        if torrent['type'] == 'file':
            ti = lt.torrent_info(torrent['data'])
            params = {"save_path": save_path, "ti": ti}
            handle = ses.add_torrent(params)
        else:  # magnet
            params = {"save_path": save_path}
            handle = lt.add_magnet_uri(ses, torrent['data'], params)

        # Fetch metadata
        start_time = time.time()
        while not handle.has_metadata():
            elapsed = time.time() - start_time
            if elapsed > METADATA_TIMEOUT:
                print(f"⚠️ Timeout while fetching metadata for: {torrent['name']}")
                ses.remove_torrent(handle)
                break
            time.sleep(1)

        if handle.has_metadata():
            info = handle.get_torrent_info()
            handle.pause()
            # Wait a moment for pause to take effect
            time.sleep(2)
            valid_torrent_infos.append((index, info, handle))
            print(f"✅ Metadata fetched for: {info.name()}")

    except Exception as e:
        print(f"❌ Error processing torrent {torrent['name']}: {e}")

print(f"\nFinished processing. Found {len(valid_torrent_infos)} valid torrent(s) with metadata.")

total_valid = len(valid_torrent_infos)

if total_valid == 0:
    print("❌ No valid torrents with metadata. Cannot continue.")
    print("Please restart the runtime and try again with new torrents.")
else:
    print(f"✅ {total_valid} torrent(s) with metadata ready.")

# Menu functions
def change_save_path():
    print("\nExplanation:")
    print("Enter the path to the folder where you want to save the downloads, relative to the root of your Google Drive's 'My Drive' folder.")
    print("For example:")
    print("- Enter '/Torrent Downloads' to save in a folder named 'Torrent Downloads' inside 'My Drive'.")
    print("- Enter '/Torrent Downloads/Movies' to save in a subfolder 'Movies' inside 'Torrent Downloads'.")
    print("- Enter '/' to save directly in the root of your 'My Drive' folder.")
    print("If you leave it empty, the default path will be used.")

    global save_path
    save_path_input = input("Enter the new save path: ").strip()
    if not save_path_input:
        print(f"No path entered, using current save path: {save_path}")
    else:
        # Ensure the path starts with '/'
        if not save_path_input.startswith('/'):
            save_path_input = '/' + save_path_input
        save_path = f"/content/drive/My Drive{save_path_input}"
        print(f"Using custom save path: {save_path}")

        # Update save path for all existing handles
        for _, _, handle in valid_torrent_infos:
            handle.move_storage(save_path)

def display_torrent_info(info, idx):
    print(f"\n📦 Torrent {idx + 1}: {info.name()}")
    f_list = info.files()
    for j in range(f_list.num_files()):
        f = f_list.file_path(j)
        sz = f_list.file_size(j)
        print(f"   📄 {f} - {sz / (1024 ** 2):.2f} MB")
    size = sum([f_list.file_size(j) for j in range(f_list.num_files())])
    return size

def view_torrent_metadata():
    try:
        torrent_index = int(input(f"Enter the number of the torrent to view (1-{len(valid_torrent_infos)}): ")) - 1
        if 0 <= torrent_index < len(valid_torrent_infos):
            _, info, _ = valid_torrent_infos[torrent_index]
            display_torrent_info(info, torrent_index)
        else:
            print("❌ Invalid torrent number.")
    except ValueError:
        print("❌ Invalid input. Please enter a number.")

def remove_torrent():
    global valid_torrent_infos

    if len(valid_torrent_infos) == 0:
        print("❌ No torrents to remove.")
        return

    try:
        torrent_index = int(input(f"Enter the number of the torrent to remove (1-{len(valid_torrent_infos)}): ")) - 1
        if 0 <= torrent_index < len(valid_torrent_infos):
            _, info, handle = valid_torrent_infos[torrent_index]
            torrent_name = info.name()

            # Confirm removal
            confirm = input(f"⚠️ Are you sure you want to remove '{torrent_name}'? (yes/no): ").strip().lower()
            if confirm == 'yes':
                # Remove from libtorrent session
                ses.remove_torrent(handle)

                # Remove from our list
                valid_torrent_infos.pop(torrent_index)

                print(f"✅ Successfully removed torrent: {torrent_name}")

                if len(valid_torrent_infos) == 0:
                    print("⚠️ No torrents remaining in the list.")
            else:
                print("❌ Removal cancelled.")
        else:
            print("❌ Invalid torrent number.")
    except ValueError:
        print("❌ Invalid input. Please enter a number.")

def reorder_torrents():
    global valid_torrent_infos

    if len(valid_torrent_infos) <= 1:
        print("❌ Need at least 2 torrents to reorder.")
        return

    reorder_input = input(f"\n🔀 Enter new order of torrents (space-separated numbers 1-{len(valid_torrent_infos)}), or press Enter to keep order:\n> ")

    if reorder_input.strip():
        try:
            new_order = list(map(int, reorder_input.strip().split()))
            if sorted(new_order) != list(range(1, len(valid_torrent_infos) + 1)):
                raise ValueError("Invalid order sequence")

            # Reorder the list (this only affects download order, not session)
            valid_torrent_infos = [valid_torrent_infos[i - 1] for i in new_order]
            print("✅ Order updated.")

        except Exception as e:
            print(f"❌ Invalid input: {e}. Keeping original order.")

def change_timeout():
    global DEFAULT_TIMEOUT
    try:
        timeout_input = input(f"⏱️ Set timeout for stuck torrents (in seconds) [Default: {DEFAULT_TIMEOUT}]: ").strip()
        if timeout_input:
            DEFAULT_TIMEOUT = int(timeout_input)

        print(f"\n✅ Setting applied:")
        print(f"- Timeout per torrent: {DEFAULT_TIMEOUT} seconds")

    except ValueError:
        print("⚠️ Invalid input, using default settings.")

def change_askaftereach():
    global ASK_AFTER_EACH
    try:
        ask_input = input("🧭 Ask for confirmation after each download? (yes/no) [Default: no]: ").strip().lower()
        ASK_AFTER_EACH = (ask_input == "yes")

        print(f"\n✅ Setting applied:")
        print(f"- Ask after each download: {'Yes' if ASK_AFTER_EACH else 'No'}")

    except ValueError:
        print("⚠️ Invalid input, using default settings.")

def print_current_settings():
    global valid_torrent_infos, free_gb, save_path, DEFAULT_TIMEOUT, ASK_AFTER_EACH

    print("\n" + "="*60)
    print("📋 CURRENT SETTINGS & STATUS")
    print("="*60)

    # Save path
    print(f"📁 Save Path: {save_path}")

    # Valid torrents list
    print(f"\n📦 Valid/Selected Torrents ({len(valid_torrent_infos)}):")
    if len(valid_torrent_infos) > 0:
        for display_idx, (_, info, _) in enumerate(valid_torrent_infos):
            print(f"   {display_idx + 1}. {info.name()}")
    else:
        print("   No torrents in list")

    # Calculate total download size and check space
    if len(valid_torrent_infos) > 0:
        total_download_bytes = sum([
            sum([info.files().file_size(i) for i in range(info.files().num_files())])
            for _, info, _ in valid_torrent_infos
        ])
        total_download_gb = total_download_bytes / (1024 ** 3)

        print(f"\n💾 Storage Info:")
        print(f"   Total Download Size: {total_download_gb:.2f} GB")
        print(f"   Google Drive Free Space: {free_gb:.2f} GB")

        if total_download_gb > free_gb:
            print("   ❌ WARNING: Not enough space on Google Drive!")
        else:
            print(f"   ✅ Space Available: {free_gb - total_download_gb:.2f} GB remaining after download")
    else:
        print(f"\n💾 Storage Info:")
        print(f"   Google Drive Free Space: {free_gb:.2f} GB")
        print("   No torrents to calculate size for")

    # Other settings
    print(f"\n⚙️ Other Settings:")
    print(f"   Timeout: {DEFAULT_TIMEOUT} seconds")
    print(f"   Ask After Each Download: {'Yes' if ASK_AFTER_EACH else 'No'}")
    print("="*60)

# Main menu loop
if total_valid > 0:
    while True:
        print_current_settings()

        print("\n--- Main Menu ---")
        print("1. Confirm Settings and Exit Menu")
        print("2. Change Save Path")
        print("3. View Torrent Metadata")
        print("4. Remove Torrent from List")
        print("5. Reorder Torrents")
        print("6. Change Timeout Setting")
        print("7. Change Ask After Each Download Setting")

        choice = input("Enter your choice: ").strip()

        if choice == '1':
            if len(valid_torrent_infos) > 0:
                print("Exiting menu. Run next cell to download")
            else:
                print("❌ No torrents remaining. Restart runtime and add torrents again.")
            break
        elif choice == '2':
            change_save_path()
        elif choice == '3':
            view_torrent_metadata()
        elif choice == '4':
            remove_torrent()
        elif choice == '5':
            reorder_torrents()
        elif choice == '6':
            change_timeout()
        elif choice == '7':
            change_askaftereach()
        else:
            print("Invalid choice. Please try again.")


## 🔹 Cell 4: Downloading Torrents

 Now the actual downloading begins! 🎉  

 - Progress bars will show how much has been downloaded.  
 - Just **keep the Colab runtime connected** at all times while this section runs. If the runtime disconnects, the download will stop.  

 ### ⚠️ Things to note:  
 - The torrents will initially be listed as **Torrent _i_** in the progress bar. The names will appear after the previous torrent download finishes.
 - If you enabled the **“Ask After Each Download”** setting, you'll need to check the notebook and answer when prompted before continuing.  
 - If a torrent takes too long, the program may stop it after the **timeout period** (default: 10 minutes). You can choose to **skip** or **wait longer**.  
 - Once all downloads finish, wait a little while for the files to fully appear in your Google Drive.  

 Sit back and relax while your torrents download straight into your Drive! 🚀  

In [ ]:
state_str = [
    "queued", "checking", "downloading metadata", "downloading",
    "finished", "seeding", "allocating", "checking fastresume"
]

layout = widgets.Layout(width="auto")
style = {"description_width": "initial"}

# Create progress bars for active torrents only
download_bars = [
    widgets.FloatSlider(
        value=0.0, min=0.0, max=100.0, step=0.01,
        description=f"Torrent {i+1}", disabled=True,
        layout=layout, style=style
    )
    for i in range(len(valid_torrent_infos))
]
display(*download_bars)

# Download torrents in the order they appear in valid_torrent_infos
for index, (original_idx, info, handle) in enumerate(valid_torrent_infos):
    name = info.name()
    print(f"\n🚀 Starting download {index + 1}/{len(valid_torrent_infos)}: {name}")
    bar = download_bars[index]

    # Resume only this torrent
    handle.resume()
    start_time = time.time()
    stuck_start_time = None  # Track when download became stuck
    skipped = False

    # Wait for torrent to start (it might need to check files first)
    print("Initializing download...")
    while True:
        s = handle.status()
        if s.state == 1 or s.state == 3 or s.state == 4:  # checking, downloading, or finished
            break
        time.sleep(1)

    # Monitor download progress
    while not handle.is_seed():
        s = handle.status()
        progress = s.progress * 100
        bar.value = progress

        # Update progress bar description with more info
        download_rate = s.download_rate / 1000  # Convert to kB/s
        state_name = state_str[s.state] if s.state < len(state_str) else "unknown"
        bar.description = f"{name[:30]} | {download_rate:.1f} kB/s | {state_name} | {progress:.1f}%"

        current_time = time.time()

        # Check if download is stuck (< 10 kB/s) only when actively downloading
        if s.state == 3:  # downloading state
            if download_rate < 10:  # Less than 10 kB/s
                if stuck_start_time is None:
                    stuck_start_time = current_time
                    print(f"⚠️ Download speed dropped below 10 kB/s, monitoring for timeout...")
                elif current_time - stuck_start_time > DEFAULT_TIMEOUT:
                    print(f"\n⏰ Download stuck for {DEFAULT_TIMEOUT} seconds (< 10 kB/s). Skipping '{name}'.")
                    skipped = True
                    break
            else:
                # Reset stuck timer if speed is good
                if stuck_start_time is not None:
                    print(f"✅ Download speed recovered ({download_rate:.1f} kB/s)")
                stuck_start_time = None
        else:
            # Reset stuck timer if not in downloading state
            stuck_start_time = None

        time.sleep(1)

    # Handle completion or skipping
    if skipped:
        handle.pause()  # Pause instead of removing to avoid issues
        print(f"⏭️ Skipped: {name}")
    else:
        bar.value = 100
        bar.description = f"{name[:30]} | Complete!"
        print(f"✅ Download complete: {name}")

        # Pause the completed torrent to free up resources
        handle.pause()

    # Ask user whether to continue (if enabled)
    if ASK_AFTER_EACH and index < len(valid_torrent_infos) - 1:
        cont = input("Continue to next download? (yes/no): ").strip().lower()
        if cont != "yes":
            print("🛑 Stopping remaining downloads.")
            # Pause all remaining torrents
            for remaining_idx in range(index + 1, len(valid_torrent_infos)):
                _, _, remaining_handle = valid_torrent_infos[remaining_idx]
                remaining_handle.pause()
            break

print("\n🎉 All downloads completed!")

# Clean up: Remove all torrent handles from session
print("Cleaning up torrent session...")
for _, _, handle in valid_torrent_infos:
    try:
        ses.remove_torrent(handle)
    except:
        pass  # Handle might already be removed